In [ ]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

# Initialize pygame mixer
mixer.init()
# Load the alarm sound
sound = mixer.Sound('alarm.wav')

# Load pre-trained Haar cascade classifiers for face and eyes detection
face = cv2.CascadeClassifier('haar cascade files/haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haar cascade files/haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haar cascade files/haarcascade_righteye_2splits.xml')

# Define labels for eye states
lbl=['Close','Open']

# Load CNN model for eye state classification
model = load_model('models/cnncat2.h5')

# Get current working directory
path = os.getcwd()

# Initialize video capture from default camera
cap = cv2.VideoCapture(0)

# Set font for text display
font = cv2.FONT_HERSHEY_COMPLEX_SMALL

# Initialize variables for counting, scoring, and eye predictions
count=0
score=0
rpred=[99]
lpred=[99]

# Main loop for video processing
while(True):
    # Read a frame from the video stream
    ret, frame = cap.read()
    height, width = frame.shape[:2]
    
    # Convert frame to grayscale for face and eyes detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale frame
    faces = face.detectMultiScale(gray, minNeighbors=5, scaleFactor=1.1, minSize=(25,25))
    
    # Detect left and right eyes in the grayscale frame
    left_eye = leye.detectMultiScale(gray)
    right_eye = reye.detectMultiScale(gray)
    
    # Draw rectangle for score display
    cv2.rectangle(frame, (0, height-50), (200, height), (0, 0, 0), thickness=cv2.FILLED)
    
    # Iterate through detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (100, 100, 100), 1)
        
    # Iterate through detected right eyes
    for (x, y, w, h) in right_eye:
        r_eye = frame[y:y+h, x:x+w]
        count += 1
        r_eye = cv2.cvtColor(r_eye, cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye, (24, 24))
        r_eye = r_eye / 255
        r_eye = r_eye.reshape(24, 24, -1)
        r_eye = np.expand_dims(r_eye, axis=0)
        rpred = model.predict_classes(r_eye)
        if rpred[0] == 1:
            lbl = 'Open'
        if rpred[0] == 0:
            lbl = 'Closed'
        break
    
    # Iterate through detected left eyes
    for (x, y, w, h) in left_eye:
        l_eye = frame[y:y+h, x:x+w]
        count += 1
        l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2GRAY)
        l_eye = cv2.resize(l_eye, (24, 24))
        l_eye = l_eye / 255
        l_eye = l_eye.reshape(24, 24, -1)
        l_eye = np.expand_dims(l_eye, axis=0)
        lpred = model.predict_classes(l_eye)
        if lpred[0] == 1:
            lbl = 'Open'
        if lpred[0] == 0:
            lbl = 'Closed'
        break
    
    # Update score based on eye state predictions
    if rpred[0] == 0 and lpred[0] == 0:
        score += 1
        cv2.putText(frame, "Closed", (10, height-20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
    else:
        score -= 1
        cv2.putText(frame, "Open", (10, height-20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
    
    # Ensure score is non-negative
    if score < 0:
        score = 0
    
    # Display current score
    cv2.putText(frame, 'Score:' + str(score), (100, height-20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
    
    # Check if person is drowsy and trigger alarm
    if score > 15:
        # Save image of drowsy person
        cv2.imwrite(os.path.join(path, 'image.jpg'), frame)
        try:
            # Play alarm sound
            sound.play()
        except:
            pass
    
    # Adjust rectangle thickness based on score
    thicc = 2 if thicc < 2 else thicc
    cv2.rectangle(frame, (0, 0), (width, height), (0, 0, 255), thicc)
    
    # Display the processed frame
    cv2.imshow('frame', frame)
    
    # Check for quit key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()
